In [13]:
#Code Written by Diana Lewis 
import cv2
import os
import csv
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [12]:
# Function to overlay segmented masks on images
def overlay_masks(image, cup_mask, disk_mask):
    overlay = image.copy()
    overlay[cup_mask != 0] = [0, 255, 0]  # Green color for cup mask
    overlay[disk_mask != 0] = [0, 0, 255]  # Red color for disk mask
    return cv2.addWeighted(overlay, 0.5, image, 0.5, 0)

In [6]:
# Load ground truth classifications from CSV file
ground_truth_dict = {}
csv_file = "Glaucoma_Balanced_Dataset/Preprocessed_images_input_to_bound/bounding_box_labels_adjusted_CDR_CSV.csv"
with open(csv_file, mode='r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip header row
    for row in reader:
        if row[5] == 'NRG':
            ground_truth_dict[row[0]] = 0  # Not Glaucoma
        elif row[5] == 'RG':
            ground_truth_dict[row[0]] = 1  # Glaucoma


In [11]:
# Function to calculate optic cup and optic disk diameters from segmented masks
#Need to call from segmented_cup_mask and segmented_disk_mask from ROI-FCN-GNET-Use-This.ipynb
def calculate_diameters(segmented_cup_mask, segmented_disk_mask):
    # Calculate the contour of the segmented cup mask
    _, cup_contours, _ = cv2.findContours(segmented_cup_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Calculate the contour of the segmented disk mask
    _, disk_contours, _ = cv2.findContours(segmented_disk_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find the largest contour for optic cup
    cup_contour = max(cup_contours, key=cv2.contourArea)
    # Find the largest contour for optic disk
    disk_contour = max(disk_contours, key=cv2.contourArea)
    
    # Calculate the diameter of optic cup
    _, _, cup_width, cup_height = cv2.boundingRect(cup_contour)
    cup_diameter = max(cup_width, cup_height)
    
    # Calculate the diameter of optic disk
    _, _, disk_width, disk_height = cv2.boundingRect(disk_contour)
    disk_diameter = max(disk_width, disk_height)
    
    return cup_diameter, disk_diameter

# Calculate cup and disk diameters and CDR values
optic_cup_diameters = []
optic_disk_diameters = []
for cup_mask, disk_mask in zip(segmented_cup_masks, segmented_disk_masks):
    cup_diameter, disk_diameter = calculate_diameters(cup_mask, disk_mask)
    optic_cup_diameters.append(cup_diameter)
    optic_disk_diameters.append(disk_diameter)

# Convert lists to numpy arrays
X = np.array(X)  # Assuming X is a list of images
optic_cup_diameters = np.array(optic_cup_diameters)
optic_disk_diameters = np.array(optic_disk_diameters)

# Calculate CDR
#cdr_values = [cup_diameter / disk_diameter for cup_diameter, disk_diameter in zip(optic_cup_diameters, optic_disk_diameters)]

# Calculate CDR
cdr_values = optic_cup_diameters / optic_disk_diameters

# Convert CDR values to binary labels
y = np.where(cdr_values > 0.65, 1, 0)

# Convert CDR values to binary labels
# y = [1 if cdr > 0.65 else 0 for cdr in cdr_values]


NameError: name 'segmented_cup_masks' is not defined

In [8]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val, cdr_train, cdr_val = train_test_split(X, y, cdr_values, test_size=0.2, random_state=42)

# Preprocess the images and overlay the segmentation masks
def preprocess_image(image, cup_mask, disk_mask):
    Resize the image and masks
    image = cv2.resize(image, (256, 256))
    cup_mask = cv2.resize(cup_mask, (224, 224))
    disk_mask = cv2.resize(disk_mask, (224, 224))
    #Overlay the masks on the image
    return overlay_masks(image, cup_mask, disk_mask)

X_train = [preprocess_image(X_train[i], segmented_cup_masks[i], segmented_disk_masks[i]) for i in range(len(X_train))]
X_val = [preprocess_image(X_val[i], segmented_cup_masks[i], segmented_disk_masks[i]) for i in range(len(X_val))]

# Load MobileNetV3Small model
base_model = tf.keras.applications.MobileNetV3Small(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom classification head
input_image = tf.keras.Input(shape=(224, 224, 3))
base_model_output = base_model(input_image)
global_avg_pooling_output = tf.keras.layers.GlobalAveragePooling2D()(base_model_output)
cdr_input = tf.keras.Input(shape=(1,))
concatenated_input = tf.keras.layers.Concatenate()([global_avg_pooling_output, cdr_input])
dense_output = tf.keras.layers.Dense(1, activation='sigmoid')(concatenated_input)
model = tf.keras.Model(inputs=[input_image, cdr_input], outputs=dense_output)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit([np.array(X_train), cdr_train], y_train, epochs=10, validation_data=([np.array(X_val), cdr_val], y_val))

# Make predictions
y_pred = model.predict([np.array(X_val), cdr_val])
y_pred = (y_pred > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

# Compare predicted CDR values with ground truth classifications
for i in range(len(y_val)):
    image_name = os.path.basename(X_val[i])  # Assuming X_val contains image paths
    ground_truth = ground_truth_dict.get(image_name, "Unknown")
    print(f"Image: {image_name}, Ground Truth: {ground_truth}, Predicted CDR: {y_pred[i]}")

NameError: name 'dataset' is not defined